[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/repos-especializacion-UdeA/monografia_modelos/blob/main/03_features_extraction.ipynb)

# Extracción de caracteristicas

In [2]:
try:
    import google.colab 
    !wget --no-cache -O init.py -q https://raw.githubusercontent.com/repos-especializacion-UdeA/monografia_modelos/refs/heads/main/init.py
    from init import init; init(force_download=False)
except ImportError:
    print("Ejecucion del notebook en entorno local")

Ejecucion del notebook en entorno local


## Objetivo a desarrollar

To do...

## Actividades

To Do...

## 1. Librerias y configuraciones previas


In [1]:
# Verificacion de librerias necesarias antes de empezar
try:
    import scipy.io
    print("La librería 'scipy' está instalada y se ha importado correctamente.")
except ImportError:
    print("La librería 'scipy' no está instalada.")
    print("Instalando 'scipy'...")
    !pip install scipy

try:
    import libemg
    print("La librería 'libEMG' está instalada y se ha importado correctamente.")
except ImportError:
    print("La librería 'libEMG' no está instalada.")
    print("Instalando 'libEMG'...")
    !pip install libemg

try:
    import imblearn
    print("La librería 'imblearn' está instalada y se ha importado correctamente.")
except ImportError:
    print("La librería 'imblearn' no está instalada.")
    print("Instalando 'imblearn'...")
    !pip install imblearn

La librería 'scipy' está instalada y se ha importado correctamente.
La librería 'libEMG' está instalada y se ha importado correctamente.
La librería 'imblearn' está instalada y se ha importado correctamente.


In [ ]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Almacenar en caché los resultados de funciones en el disco
# ==============================================================================
import joblib


# Gestion de librerias
# ==============================================================================
from importlib import reload

# Matemáticas y estadísticas
# ==============================================================================
import math

# Preparación de datos
# ==============================================================================
from imblearn.over_sampling import RandomOverSampler
from sklearn.neighbors import LocalOutlierFactor

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns


# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Formateo y estilo
# ==============================================================================
from IPython.display import Markdown, display

# Biblioteca scipy y componentes
# ==============================================================================
import scipy.io
from scipy import signal

# Bliblioteca de archivos
# ==============================================================================
import requests
from tqdm import tqdm
import os
import zipfile
import sys
import copy

# Biblioteca libemg
# ==============================================================================
from libemg.data_handler import OfflineDataHandler, RegexFilter
from libemg.utils import get_windows
from libemg.feature_extractor import FeatureExtractor
from libemg import filtering

## 2. Funciones

In [5]:
def find_value_ranges(arr, value):
    """
    Encuentra los rangos contiguos donde un valor específico aparece en un arreglo.

    Este método recorre el arreglo de entrada y detecta bloques consecutivos 
    donde el valor coincide con el especificado, retornando las posiciones 
    de inicio y fin de cada uno de esos bloques.

    Parameters
    ----------
    arr : array-like
        Arreglo de entrada, puede ser una lista o un `numpy.ndarray`.
    value : int or float
        Valor que se desea buscar de forma contigua en el arreglo.

    Returns
    -------
    list of tuple of int
        Lista de tuplas (inicio, fin) indicando los índices de cada bloque contiguo 
        donde `value` aparece. Los índices son inclusivos.

    Examples
    --------
    >>> find_value_ranges([1, 2, 2, 3, 2, 2, 2, 4], 2)
    [(1, 2), (4, 6)]

    >>> find_value_ranges([5, 5, 5, 1, 2, 3], 5)
    [(0, 2)]
    """
    arr = np.asarray(arr)
    ranges = []
    in_range = False

    for i, val in enumerate(arr):
        if val == value and not in_range:
            start = i
            in_range = True
        elif val != value and in_range:
            end = i - 1
            ranges.append((start, end))
            in_range = False

    # Si termina en un bloque del valor
    if in_range:
        ranges.append((start, len(arr) - 1))

    return ranges

In [12]:
def concatenar_matrices_con_clave(diccionario_matrices):
    """
    Concatena verticalmente matrices almacenadas en un diccionario,
    agregando una columna con el valor de la clave como etiqueta.

    Para cada par (clave, matriz) en el diccionario, se antepone una columna que contiene 
    el valor entero de la clave repetido en cada fila. Las matrices resultantes se concatenan 
    verticalmente en una sola matriz final.

    Parameters
    ----------
    diccionario_matrices : dict of str to numpy.ndarray
        Diccionario cuyas claves son cadenas de dígitos (por ejemplo: '1', '2', '3') y cuyos
        valores son matrices NumPy de igual número de columnas. Cada matriz representa un bloque 
        de datos asociado a una clase o grupo distinto.

    Returns
    -------
    numpy.ndarray
        Matriz resultante de concatenar todas las matrices, con una columna adicional al principio
        que contiene la etiqueta de clase correspondiente a cada fila (según la clave del diccionario).

    Raises
    ------
    ValueError
        Si alguna matriz no tiene el mismo número de columnas que las demás.

    Examples
    --------
    >>> d = {
    ...     '1': np.array([[1, 2], [3, 4]]),
    ...     '2': np.array([[5, 6]])
    ... }
    >>> concatenar_matrices_con_clave(d)
    array([[1, 1, 2],
           [1, 3, 4],
           [2, 5, 6]])
    """
    matrices_modificadas = []
    n_columnas = None

    for k, matriz in diccionario_matrices.items():
        # Validar que todas las matrices tengan el mismo número de columnas
        if n_columnas is None:
            n_columnas = matriz.shape[1]
        elif matriz.shape[1] != n_columnas:
            raise ValueError("Todas las matrices deben tener el mismo número de columnas.")

        clave_int = int(k)
        n_filas = matriz.shape[0]

        # Crear una columna con el valor de la clave
        columna_clave = np.full((n_filas, 1), clave_int)

        # Añadir la etiqueta como primera columna
        matriz_modificada = np.hstack((columna_clave, matriz))
        matrices_modificadas.append(matriz_modificada)

    # Concatenar todas las matrices resultantes
    return np.concatenate(matrices_modificadas, axis=0)

In [63]:
def segmentate_data_base(data, window_size, window_increment):
    """
    Segmenta múltiples conjuntos de datos en ventanas de tamaño fijo con un desplazamiento especificado.

    Para cada conjunto de datos contenido en el diccionario `data`, esta función aplica una segmentación
    basada en ventanas deslizantes. Cada ventana tiene tamaño `window_size` y se desplaza `window_increment`
    muestras entre cada segmento consecutivo.

    Parameters
    ----------
    data : dict of str to numpy.ndarray
        Diccionario donde las claves identifican subconjuntos de datos (por ejemplo, sujetos o ejercicios),
        y los valores son matrices NumPy 2D a segmentar. Cada matriz debe tener forma (n_muestras, n_características).
    window_size : int
        Tamaño (en número de muestras) de cada ventana.
    window_increment : int
        Número de muestras que avanza la ventana en cada iteración (desplazamiento).

    Returns
    -------
    dict of str to numpy.ndarray
        Diccionario con la misma estructura de claves que el original, pero donde cada valor es una matriz
        que contiene las ventanas generadas a partir del conjunto original.

    Examples
    --------
    >>> data = {
    ...     "1": np.random.rand(1000, 10),
    ...     "2": np.random.rand(1200, 10)
    ... }
    >>> segmentate_data_base(data, window_size=200, window_increment=100).keys()
    dict_keys(['1', '2'])
    """
    # Aplicar segmentación por ventanas para cada subconjunto de datos en el diccionario
    windows_data = {
        key: get_windows(subset_data, window_size, window_increment)
        for key, subset_data in data.items()
    }

    return windows_data


In [71]:
def get_features(data, feature_list, window_size=30, window_increment=20):
    """
    Extrae características de señales EMG segmentadas en ventanas y asigna etiquetas por ventana.

    Esta función realiza los siguientes pasos:
    1. Segmenta los datos por ventana.
    2. Separa los canales EMG (primeros 10) y el canal de estímulo (índice 10).
    3. Extrae las características especificadas mediante FeatureExtractor.
    4. Asigna etiquetas a cada ventana si todos los valores del estímulo son iguales.
    5. Concatena las características extraídas con sus respectivas etiquetas.

    Parameters
    ----------
    data : dict of str to numpy.ndarray
        Diccionario donde cada clave representa un grupo o sujeto, y cada valor es una matriz de datos 
        EMG con forma (n_muestras, n_canales), incluyendo el canal de estímulo en la posición 10.
    feature_list : list of str
        Lista de nombres de características a extraer, compatibles con FeatureExtractor.
    window_size : int, optional
        Tamaño de cada ventana deslizante en número de muestras (por defecto 30).
    window_increment : int, optional
        Número de muestras que avanza cada ventana (por defecto 20).

    Returns
    -------
    dict of str to numpy.ndarray
        Diccionario con los datos procesados. Cada valor es una matriz donde:
        - Las columnas corresponden a las características extraídas.
        - La última columna contiene la etiqueta (`restimulus`) por ventana.
          Si los valores de estímulo dentro de una ventana no son constantes, se marca como -1.

    Notes
    -----
    - Se asume que los primeros 10 canales corresponden a señales EMG y el canal 10 (índice 10) es el estímulo.
    - Las etiquetas por ventana solo se asignan si todos los valores del canal de estímulo son iguales.

    Examples
    --------
    >>> data = {'1': np.random.rand(1000, 11)}
    >>> features = get_features(data, ['mean', 'std'])
    >>> features['1'].shape
    (num_ventanas, num_features + 1)
    """
    # Paso 1: Segmentar los datos por ventana
    windows_data = segmentate_data_base(data, window_size, window_increment)

    emg_data = {}
    restimulus_data = {}

    # Paso 2: Separar canales EMG y canal de estímulo (canal 10)
    for key, windows in windows_data.items():
        emg_data[key] = windows[:, :10, :]      # Primeros 10 canales: señales EMG
        restimulus_data[key] = windows[:, 10, :]  # Canal 10: etiquetas de estímulo

    # Paso 3: Inicializar extractor de características
    fe = FeatureExtractor()
    features_data = {}

    # Extraer características para cada sujeto
    for key, emg in emg_data.items():
        features_data[key] = fe.extract_features(feature_list, emg)

    # Paso 4: Procesar etiquetas de estímulo por ventana
    processed_restimulus_data = {}
    for key, stim in restimulus_data.items():
        # Etiqueta válida solo si todos los valores dentro de la ventana son iguales
        processed_restimulus_data[key] = np.where(
            np.all(stim == stim[:, [0]], axis=1),
            stim[:, 0],
            -1
        )

    # Paso 5: Concatenar características y etiquetas en una sola matriz por sujeto
    concatenated_data = {}
    for key in features_data:
        feature_arrays = []
        for feature in feature_list:
            if feature in features_data[key]:
                feature_arrays.append(features_data[key][feature])
        
        labels = processed_restimulus_data[key].reshape(-1, 1)
        concatenated_data[key] = np.hstack(feature_arrays + [labels])

    return concatenated_data


In [ ]:
def save_features_to_csv(features_dict, feature_list, output_dir, label_column_name="label"):
    """
    Guarda los datos de características y etiquetas en archivos CSV, uno por sujeto.

    Cada archivo generado contiene una tabla con columnas nombradas automáticamente a partir 
    de los nombres de características proporcionados, y una columna final que contiene la etiqueta 
    por ventana (por ejemplo, clase o tipo de estímulo).

    Parameters
    ----------
    features_dict : dict of str to numpy.ndarray
        Diccionario generado por `get_features`, donde cada valor es una matriz de forma
        (n_ventanas, n_features + 1). La última columna debe ser la etiqueta por ventana.
    feature_list : list of str
        Lista de nombres de características utilizadas. Se asume que las características están
        organizadas en bloques por tipo dentro de cada ventana.
    output_dir : str
        Ruta del directorio donde se guardarán los archivos CSV.
    label_column_name : str, optional
        Nombre que se asignará a la última columna (etiqueta de clase). Por defecto: "label".

    Returns
    -------
    None
        Esta función no retorna nada. Genera archivos CSV en el directorio especificado.

    Raises
    ------
    ValueError
        Si alguna entrada del diccionario no es un arreglo NumPy válido.

    Examples
    --------
    >>> save_features_to_csv(features_dict, ['mean', 'std'], './output')
    # Crea archivos como: ./output/sujeto_1_features.csv
    """
    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    for subject_id, data_array in features_dict.items():
        # Validar que el contenido sea una matriz NumPy
        if not isinstance(data_array, np.ndarray):
            raise ValueError(f"El valor para el sujeto {subject_id} no es un array NumPy válido.")

        # Determinar el número total de características (excluyendo la etiqueta)
        n_total_features = data_array.shape[1] - 1
        n_feats_per_type = n_total_features // len(feature_list)

        # Generar nombres de columnas para cada tipo de característica
        column_names = []
        for feat in feature_list:
            for i in range(n_feats_per_type):
                column_names.append(f"{feat}_f{i}")

        # Agregar la columna de etiqueta
        column_names.append(label_column_name)

        # Crear un DataFrame y exportarlo
        df = pd.DataFrame(data_array, columns=column_names)
        filename = os.path.join(output_dir, f"sujeto_{subject_id}_features.csv")
        df.to_csv(filename, index=False)
        print(f"Guardado: {filename}")


## 3. Variables de entorno 

In [7]:
ROOT_DIR =  './'
DATA_DIR =  ROOT_DIR + 'local/data/'
if not os.path.exists(ROOT_DIR):
    os.makedirs(ROOT_DIR)    
RAW_DATA_DIR = DATA_DIR + 'raw'
if not os.path.exists(RAW_DATA_DIR):
    os.makedirs(RAW_DATA_DIR)
RAW_DATA_DIR_NPY = DATA_DIR + 'raw_numpy'
if not os.path.exists(RAW_DATA_DIR_NPY):
    os.makedirs(RAW_DATA_DIR_NPY)
RAW_SUBSET_DATA_DIR = DATA_DIR + 'raw_subset'
if not os.path.exists(RAW_SUBSET_DATA_DIR):
    os.makedirs(RAW_SUBSET_DATA_DIR)
FILTERED_SUBSET_DATA_DIR = DATA_DIR + 'filtered_subset'
if not os.path.exists(FILTERED_SUBSET_DATA_DIR):
    os.makedirs(FILTERED_SUBSET_DATA_DIR)
FEATURES_SUBSET_DATA_DIR = DATA_DIR + 'features_subset'
if not os.path.exists(FEATURES_SUBSET_DATA_DIR):
    os.makedirs(FEATURES_SUBSET_DATA_DIR)

print("Directorio de trabajo:", ROOT_DIR)
print("Directorio de datos:", DATA_DIR)
print("Directorio de datos crudos:", RAW_DATA_DIR)
print("Directorio de datos crudos (numpy):", RAW_DATA_DIR_NPY)
print("Directorio de datos crudos (subconjunto):", RAW_SUBSET_DATA_DIR)
print("Directorio de datos filtrados (subconjunto):", FILTERED_SUBSET_DATA_DIR)
print("Directorio de datos de características (subconjunto):", FEATURES_SUBSET_DATA_DIR)

Directorio de trabajo: ./
Directorio de datos: ./local/data/
Directorio de datos crudos: ./local/data/raw
Directorio de datos crudos (numpy): ./local/data/raw_numpy
Directorio de datos crudos (subconjunto): ./local/data/raw_subset
Directorio de datos filtrados (subconjunto): ./local/data/filtered_subset
Directorio de datos de características (subconjunto): ./local/data/features_subset


## 4. Carga del dataset con los datos filtrados

En nuestro analisis, solo nos va a interesar los datos de las columnas:
* **`emg`**: Señales EMG superficiales tomadas de los 10 electronos.
* **`rerepetition`**: Repetición de la postura de la mano (ejercicio) realizado. Cada postura se repetira 10 veces.
* **`restimulus`**: Postura manual (ejercicio) realizada.

Es importante anotar que los datos cargados son matrices cuyas columnas se relacionan con la información de la siguiente forma:

```
[ emg | restimulus | rerepetition ] = [ 0:9 | 10 | 11 ]
```

### 4.1. Carga y combinacion de los datos filtrados

Los datos de cada sujeto se cargan y combianan en el diccionario `raw_data`

In [8]:
# Listar y cargar los archivos .npy del directorio PROCESSED_DATA_DIR
npy_files = [f for f in os.listdir(FILTERED_SUBSET_DATA_DIR) if f.endswith('.npy')]

data = {}
for file in npy_files:
    file_path = os.path.join(FILTERED_SUBSET_DATA_DIR, file)
    sub_i = file.split('_s')[-1].split('.')[0]
    sub_i = sub_i.split('_')[0]
    data[sub_i] = np.load(file_path)
    print(f"Archivo cargado: {file}")

Archivo cargado: filtered_subset_s10_p1_23_E3.npy
Archivo cargado: filtered_subset_s11_p1_23_E3.npy
Archivo cargado: filtered_subset_s12_p1_23_E3.npy
Archivo cargado: filtered_subset_s13_p1_23_E3.npy
Archivo cargado: filtered_subset_s14_p1_23_E3.npy
Archivo cargado: filtered_subset_s15_p1_23_E3.npy
Archivo cargado: filtered_subset_s16_p1_23_E3.npy
Archivo cargado: filtered_subset_s17_p1_23_E3.npy
Archivo cargado: filtered_subset_s18_p1_23_E3.npy
Archivo cargado: filtered_subset_s19_p1_23_E3.npy
Archivo cargado: filtered_subset_s1_p1_23_E3.npy
Archivo cargado: filtered_subset_s20_p1_23_E3.npy
Archivo cargado: filtered_subset_s21_p1_23_E3.npy
Archivo cargado: filtered_subset_s22_p1_23_E3.npy
Archivo cargado: filtered_subset_s23_p1_23_E3.npy
Archivo cargado: filtered_subset_s24_p1_23_E3.npy
Archivo cargado: filtered_subset_s25_p1_23_E3.npy
Archivo cargado: filtered_subset_s26_p1_23_E3.npy
Archivo cargado: filtered_subset_s27_p1_23_E3.npy
Archivo cargado: filtered_subset_s2_p1_23_E3.npy
Ar

Se inspecciona la carga de los datos

In [9]:
len(data)  # Mostrar la cantidad de archivos cargados

27

## 5. Extracción de caracteristicas

### 5.1. Concatenación de todos los datos en un unico dataframe

Se pasan todos los datos del diccionario en el que se cargaron los datos  (`raw_data`) a un unica matriz para facilitar la analitica de estos. Esta matriz tendra como primera columna el numero del sujeto, las columnas de esta matriz seran:


```
[ subject | emg | restimulus | rerepetition ] = [ 0 | 1:10 | 11 | 12 ]
```

Invocación de la función

In [13]:
all_users_data = concatenar_matrices_con_clave(data)
all_users_data.shape  # Mostrar la forma de los datos concatenados

(432383, 13)

Se seleccinan de manera aleatoria 5 muestras para verificar consistencia de la matrix:

In [15]:
# Seleccionar una muestra aleatoria de 20 filas
SAMPLES = 5
sample_data = all_users_data[np.random.choice(all_users_data.shape[0], SAMPLES, replace=False)]

# Mostrar la muestra seleccionada
print(sample_data)

[[1.50000000e+01 2.47206765e-01 1.84232274e-02 2.56200664e-03
  2.42554418e-03 2.40346181e-03 2.41145237e-03 2.36958153e-03
  2.36452485e-01 2.40442829e-03 7.71330627e-02 0.00000000e+00
  0.00000000e+00]
 [1.40000000e+01 6.80642527e-02 2.43229788e-03 2.42544982e-03
  2.33120610e-03 2.50350455e-02 1.17995347e-01 2.96463241e-02
  7.82610172e-02 7.09972797e-02 4.56073390e-03 0.00000000e+00
  0.00000000e+00]
 [6.00000000e+00 4.35673413e-01 1.12566527e-01 2.56489143e-01
  8.18206842e-02 1.87314992e-02 6.53236696e-02 8.28631178e-01
  6.25170503e-01 2.71146532e-01 7.14972621e-01 1.00000000e+00
  2.00000000e+00]
 [2.50000000e+01 3.60193876e-03 9.94904726e-01 5.13089598e-01
  1.90411301e-01 3.34401486e-01 2.36605412e+00 4.11782138e+00
  1.56906757e+00 2.49840907e-01 6.50947659e-01 2.30000000e+01
  9.00000000e+00]
 [3.00000000e+00 4.42247557e-01 6.42964899e-01 9.10987993e-01
  2.24553436e-02 2.84582707e-03 3.83436322e-03 9.28606750e-02
  5.63098221e-01 5.09699640e-02 1.13787911e-01 2.30000000e+0

In [18]:
# Crear un DataFrame a partir de all_users_raw_data
columns = ['subject'] + [f'emg_{i}' for i in range(1, 11)] + ['restimulus', 'rerepetition']
df_all_users = pd.DataFrame(all_users_data, columns=columns)

# Mostrar las primeras filas del DataFrame
df_all_users.head()

,subject,emg_1,emg_2,emg_3,emg_4,emg_5,emg_6,emg_7,emg_8,emg_9,emg_10,restimulus,rerepetition
0,10.0,0.247590,0.101654,-0.004054,0.002397,0.002365,0.002183,-0.005319,-0.027150,0.002326,-0.021768,0.0,0.0
1,10.0,0.242329,0.098765,-0.004391,0.002396,0.002362,0.002181,-0.005193,-0.027887,0.002318,-0.022504,0.0,0.0
2,10.0,0.237068,0.095835,-0.004730,0.002395,0.002359,0.002180,-0.005026,-0.028523,0.002309,-0.023206,0.0,0.0
3,10.0,0.231825,0.092875,-0.005072,0.002394,0.002356,0.002179,-0.004812,-0.029049,0.002300,-0.023867,0.0,0.0
4,10.0,0.226615,0.089892,-0.005416,0.002392,0.002353,0.002180,-0.004550,-0.029451,0.002291,-0.024482,0.0,0.0


### 5.2. Descripción general del dataframe

In [19]:
df_all_users.shape  # Mostrar la forma de los datos concatenados

(432383, 13)

In [20]:
df_all_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432383 entries, 0 to 432382
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject       432383 non-null  float64
 1   emg_1         432383 non-null  float64
 2   emg_2         432383 non-null  float64
 3   emg_3         432383 non-null  float64
 4   emg_4         432383 non-null  float64
 5   emg_5         432383 non-null  float64
 6   emg_6         432383 non-null  float64
 7   emg_7         432383 non-null  float64
 8   emg_8         432383 non-null  float64
 9   emg_9         432383 non-null  float64
 10  emg_10        432383 non-null  float64
 11  restimulus    432383 non-null  float64
 12  rerepetition  432383 non-null  float64
dtypes: float64(13)
memory usage: 42.9 MB


Como hay columnas que tienen que ver con datos categoricos y el dataframe original tiene todos los datos numericos, se hace la conversión de las columnas categoricas cargadas como numericas a datos categoricos.

In [21]:
# Convertir las columnas en categóricas
df_all_users['subject'] = df_all_users['subject'].astype('category')
df_all_users['restimulus'] = df_all_users['restimulus'].astype('category')
df_all_users['rerepetition'] = df_all_users['rerepetition'].astype('category')

# Verificar los cambios
df_all_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432383 entries, 0 to 432382
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   subject       432383 non-null  category
 1   emg_1         432383 non-null  float64 
 2   emg_2         432383 non-null  float64 
 3   emg_3         432383 non-null  float64 
 4   emg_4         432383 non-null  float64 
 5   emg_5         432383 non-null  float64 
 6   emg_6         432383 non-null  float64 
 7   emg_7         432383 non-null  float64 
 8   emg_8         432383 non-null  float64 
 9   emg_9         432383 non-null  float64 
 10  emg_10        432383 non-null  float64 
 11  restimulus    432383 non-null  category
 12  rerepetition  432383 non-null  category
dtypes: category(3), float64(10)
memory usage: 34.2 MB


### 5.3. Segmentación

La segmentación consiste en extraer porciones de la señal mediante ventanas de tiempo (que pueden ir sobrelapadas o no) para realizar una analisis local mas focalizado. La siguiente figura resume el procedimiento:

![overlap_win](overlap_window.png)

La segmentación se realiza tomando ventanas de 30 muestras de tamaño con un sobrelapamiento de 10 muestras. Antes de intentar generalizar el procedimiento, vamos a realizar la prueba con los datos asociados al primer sujeto:

```py
INDEX_SUB = '1'
emg_s1 = filtered_data[INDEX_SUB][:,0:10]
restimulus_s1 = filtered_data[INDEX_SUB][:,10]
rerepetition_s1 = filtered_data[INDEX_SUB][:,11]
```

In [23]:
INDEX_SUB = '1'
data[INDEX_SUB].shape

(16036, 12)

In [24]:
# Mostrar las primeras 10 filas de los datos filtrados para el sujeto especificado en INDEX_SUB
data[INDEX_SUB][:10]

array([[-1.10901571e-03,  3.35777718e-04,  2.29754608e-03,
         2.41791565e-03,  2.46656592e-03,  1.17058176e-02,
         1.57929216e-01,  1.22639163e-02,  1.38498726e-03,
         4.04043369e-02,  0.00000000e+00,  0.00000000e+00],
       [-1.04112299e-03,  2.49932580e-04,  2.28240912e-03,
         2.42312377e-03,  2.49875419e-03,  1.23512526e-02,
         1.60794013e-01,  1.24751146e-02,  1.31715902e-03,
         4.16480950e-02,  0.00000000e+00,  0.00000000e+00],
       [-9.61654668e-04,  1.64989863e-04,  2.26635120e-03,
         2.42849756e-03,  2.53352570e-03,  1.30167117e-02,
         1.63716897e-01,  1.26969245e-02,  1.24749988e-03,
         4.29711090e-02,  0.00000000e+00,  0.00000000e+00],
       [-8.69611219e-04,  8.13842298e-05,  2.24935306e-03,
         2.43401448e-03,  2.57097691e-03,  1.37016696e-02,
         1.66695644e-01,  1.29299952e-02,  1.17612738e-03,
         4.43763017e-02,  0.00000000e+00,  0.00000000e+00],
       [-7.63940749e-04, -4.14786429e-07,  2.2313975

In [25]:
# Split the raw EMG into windows
WINDOW_SIZE = 30  # Tamaño de la ventana
WINDOW_STEP = 20  # Paso de la ventana
windows_s1 = get_windows(data[INDEX_SUB], WINDOW_SIZE, WINDOW_STEP)
windows_s1.shape  # Mostrar la forma de las ventanas de datos

(801, 12, 30)

Segun el resultado anterior, se generaron 801 segmentos de 30 muestras para cada una de las 12 señales (`emg`, `restimulus` y `rerepetition`). Cada una de estas ventanas presentaban un solapamiento de 10 entre ellas:

In [26]:
windows_s1[0].shape

(12, 30)

Ahora se muestran los datos asociados a las primeras `3` muestras de la ventana `0`:

In [27]:
windows_s1[0,:,:3]  # 

array([[-0.00110902, -0.00104112, -0.00096165],
       [ 0.00033578,  0.00024993,  0.00016499],
       [ 0.00229755,  0.00228241,  0.00226635],
       [ 0.00241792,  0.00242312,  0.0024285 ],
       [ 0.00246657,  0.00249875,  0.00253353],
       [ 0.01170582,  0.01235125,  0.01301671],
       [ 0.15792922,  0.16079401,  0.1637169 ],
       [ 0.01226392,  0.01247511,  0.01269692],
       [ 0.00138499,  0.00131716,  0.0012475 ],
       [ 0.04040434,  0.04164809,  0.04297111],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

Como se puede ver, las dos últimas filas tienen que ver las señales asociadas al tag del ejercicio. Miremos la transpuesta para tener una idea mas clara.

In [28]:
windows_s1_T = windows_s1[0].T
windows_s1_T[0:3,:]  # Mostrar las primeras filas y columnas de la primera ventana transpuesta

array([[-0.00110902,  0.00033578,  0.00229755,  0.00241792,  0.00246657,
         0.01170582,  0.15792922,  0.01226392,  0.00138499,  0.04040434,
         0.        ,  0.        ],
       [-0.00104112,  0.00024993,  0.00228241,  0.00242312,  0.00249875,
         0.01235125,  0.16079401,  0.01247511,  0.00131716,  0.04164809,
         0.        ,  0.        ],
       [-0.00096165,  0.00016499,  0.00226635,  0.0024285 ,  0.00253353,
         0.01301671,  0.1637169 ,  0.01269692,  0.0012475 ,  0.04297111,
         0.        ,  0.        ]])

Según lo anterior, a lo que se le debe sacar la feature es a las primeras 10 filas de la ventana. A continuación se muestran las 10 señales emg para las tres primeras muestras de la ventana:

In [29]:
windows_s1[0,:10,:3]

array([[-0.00110902, -0.00104112, -0.00096165],
       [ 0.00033578,  0.00024993,  0.00016499],
       [ 0.00229755,  0.00228241,  0.00226635],
       [ 0.00241792,  0.00242312,  0.0024285 ],
       [ 0.00246657,  0.00249875,  0.00253353],
       [ 0.01170582,  0.01235125,  0.01301671],
       [ 0.15792922,  0.16079401,  0.1637169 ],
       [ 0.01226392,  0.01247511,  0.01269692],
       [ 0.00138499,  0.00131716,  0.0012475 ],
       [ 0.04040434,  0.04164809,  0.04297111]])

In [30]:
# Extraer las primeras 10 filas de todos los segmentos
emg_s1 = windows_s1[:, :10, :]
emg_s1.shape  # Mostrar la forma de los datos EMG extraídos

(801, 10, 30)

Veamos esto para el segmento 0:

In [31]:
emg_s1[0,:,:3]

array([[-0.00110902, -0.00104112, -0.00096165],
       [ 0.00033578,  0.00024993,  0.00016499],
       [ 0.00229755,  0.00228241,  0.00226635],
       [ 0.00241792,  0.00242312,  0.0024285 ],
       [ 0.00246657,  0.00249875,  0.00253353],
       [ 0.01170582,  0.01235125,  0.01301671],
       [ 0.15792922,  0.16079401,  0.1637169 ],
       [ 0.01226392,  0.01247511,  0.01269692],
       [ 0.00138499,  0.00131716,  0.0012475 ],
       [ 0.04040434,  0.04164809,  0.04297111]])

### 5.4. Extracción de caracteristicas

##### 5.4.1. Prueba con la señal RMS

In [35]:
fe = FeatureExtractor() # Inicializar el extractor de características

Como prueba vamos a seleccionar como caracteristica a extraer el valor RMS para los datos correspondientes al sujeto 1

In [36]:
feature_list_test = ['RMS']

In [37]:
features_s1 = fe.extract_features(feature_list_test, emg_s1)
features_s1.keys()

dict_keys(['RMS'])

In [38]:
features_s1['RMS'].shape  # Mostrar la forma de las características extraídas

(801, 10)

Vemos que se tiene le valor RMS para cada uno de los canales, esto relacionado con las 801 ventanas. A continuación vamos a mostrar los valores RMS para de todas las señales sEMG para los 3 primeros segmentos:

In [39]:
features_s1['RMS'][0:3]  # Mostrar las primeras filas y columnas de las características extraídas

array([[0.00440766, 0.00042322, 0.0019679 , 0.00250594, 0.00341307,
        0.02393885, 0.20610223, 0.01844022, 0.00067767, 0.07467822],
       [0.02349061, 0.00587794, 0.00146186, 0.00262896, 0.00549564,
        0.03718825, 0.26740832, 0.04344034, 0.00141234, 0.14213795],
       [0.06138584, 0.02611376, 0.00260215, 0.00267404, 0.00702446,
        0.04432762, 0.32245441, 0.12288303, 0.01137533, 0.21592697]])

Vamos ahora con los tags

In [40]:
# Extraer las primeras 2 ultimas filas de todos los segmentos
restimulus_s1 = windows_s1[:, 10, :]
restimulus_s1.shape  # Mostrar la forma de las etiquetas extraídas

(801, 30)

A continuación, visualizamos los tags para los tres primeros segmentos (`restimulus` y `rerepetition`)

In [41]:
restimulus_s1[:2,:]  # Mostrar las primeras filas y columnas de las etiquetas extraídas

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

Con el fin de colocar aquellos segmentos donde hay un cambio de nivel en el estimulo se va a crear un array donde cada fila tiene el valor del estimulo si todas las columnas son iguales, de lo contrario -1 para indicar que en ese segmento hubo un cambio de nivel

In [42]:
processed_restimulus_s1 = np.where(np.all(restimulus_s1 == restimulus_s1[:, [0]], axis=1), restimulus_s1[:, 0], -1)

# Mostrar el resultado
processed_restimulus_s1

array([ 0.,  0., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1

In [43]:
processed_restimulus_s1.shape

(801,)

In [44]:
emg_s1.shape

(801, 10, 30)

In [45]:
# Concatenar features_s1['RMS'] y processed_restimulus_s1 como una nueva columna
concatenated_features = np.hstack((features_s1['RMS'], processed_restimulus_s1.reshape(-1, 1)))

# Mostrar la forma del resultado
print(concatenated_features.shape)

(801, 11)


In [46]:
# Crear un DataFrame a partir de concatenated_features
columns = [f'RMS_{i+1}' for i in range(features_s1['RMS'].shape[1])] + ['restimulus']
df_features = pd.DataFrame(concatenated_features, columns=columns)

# Mostrar las primeras filas del DataFrame
df_features.head()

,RMS_1,RMS_2,RMS_3,RMS_4,RMS_5,RMS_6,RMS_7,RMS_8,RMS_9,RMS_10,restimulus
0,0.004408,0.000423,0.001968,0.002506,0.003413,0.023939,0.206102,0.018440,0.000678,0.074678,0.0
1,0.023491,0.005878,0.001462,0.002629,0.005496,0.037188,0.267408,0.043440,0.001412,0.142138,0.0
2,0.061386,0.026114,0.002602,0.002674,0.007024,0.044328,0.322454,0.122883,0.011375,0.215927,-1.0
3,0.084903,0.062461,0.009608,0.002956,0.006360,0.053774,0.387859,0.289713,0.037026,0.282858,1.0
4,0.061730,0.104530,0.028248,0.005728,0.007340,0.071548,0.467802,0.534735,0.086235,0.336115,1.0


Hasta aqui, hemos logrado obtener la caracteristica asociada al valor RMS de los segmentos resaltan aquellos pertenecientes a una transición con el valor de -1. Es importante aclarar que esto solo se realizo para un solo sujeto de modo que el objetivo es obtener los resultados para todos los 27 sujetos. 

A continuación, se resume el procedimiento realizado con el fin de generalizarlo para todos los sujetos:

1. Se realiza la segmentación de los datos para cada sujeto:

In [47]:
# Aplicar get_windows a todos los sujetos en data
windows_data = {key: get_windows(data, 30, 20) for key, data in data.items()}

# Mostrar las formas de las ventanas generadas para cada sujeto
for key, windows in windows_data.items():
    print(f"Sujeto {key}: {windows.shape}")

Sujeto 10: (798, 12, 30)
Sujeto 11: (804, 12, 30)
Sujeto 12: (796, 12, 30)
Sujeto 13: (799, 12, 30)
Sujeto 14: (792, 12, 30)
Sujeto 15: (796, 12, 30)
Sujeto 16: (799, 12, 30)
Sujeto 17: (801, 12, 30)
Sujeto 18: (798, 12, 30)
Sujeto 19: (805, 12, 30)
Sujeto 1: (801, 12, 30)
Sujeto 20: (798, 12, 30)
Sujeto 21: (804, 12, 30)
Sujeto 22: (799, 12, 30)
Sujeto 23: (794, 12, 30)
Sujeto 24: (802, 12, 30)
Sujeto 25: (800, 12, 30)
Sujeto 26: (807, 12, 30)
Sujeto 27: (794, 12, 30)
Sujeto 2: (798, 12, 30)
Sujeto 3: (797, 12, 30)
Sujeto 4: (799, 12, 30)
Sujeto 5: (807, 12, 30)
Sujeto 6: (802, 12, 30)
Sujeto 7: (797, 12, 30)
Sujeto 8: (802, 12, 30)
Sujeto 9: (805, 12, 30)


2. Separar para cada sujeto las señales del sEMG y del gesto realizado como ejercicio

In [48]:
# Crear diccionarios separados para emg y restimulus
emg_data = {}
restimulus_data = {}

for key, windows in windows_data.items():
    emg_data[key] = windows[:, :10, :]  # Extraer las primeras 10 filas
    restimulus_data[key] = windows[:, 10, :]  # Extraer la fila 11

# Mostrar las claves y formas de los datos extraídos
for key in emg_data:
    print(f"Clave: {key}, EMG Shape: {emg_data[key].shape}, Restimulus Shape: {restimulus_data[key].shape}")

Clave: 10, EMG Shape: (798, 10, 30), Restimulus Shape: (798, 30)
Clave: 11, EMG Shape: (804, 10, 30), Restimulus Shape: (804, 30)
Clave: 12, EMG Shape: (796, 10, 30), Restimulus Shape: (796, 30)
Clave: 13, EMG Shape: (799, 10, 30), Restimulus Shape: (799, 30)
Clave: 14, EMG Shape: (792, 10, 30), Restimulus Shape: (792, 30)
Clave: 15, EMG Shape: (796, 10, 30), Restimulus Shape: (796, 30)
Clave: 16, EMG Shape: (799, 10, 30), Restimulus Shape: (799, 30)
Clave: 17, EMG Shape: (801, 10, 30), Restimulus Shape: (801, 30)
Clave: 18, EMG Shape: (798, 10, 30), Restimulus Shape: (798, 30)
Clave: 19, EMG Shape: (805, 10, 30), Restimulus Shape: (805, 30)
Clave: 1, EMG Shape: (801, 10, 30), Restimulus Shape: (801, 30)
Clave: 20, EMG Shape: (798, 10, 30), Restimulus Shape: (798, 30)
Clave: 21, EMG Shape: (804, 10, 30), Restimulus Shape: (804, 30)
Clave: 22, EMG Shape: (799, 10, 30), Restimulus Shape: (799, 30)
Clave: 23, EMG Shape: (794, 10, 30), Restimulus Shape: (794, 30)
Clave: 24, EMG Shape: (802

3. Obtener las caracteristicas asociadas a para cada uno de los segmentos de las señales sEMG

In [49]:
fe = FeatureExtractor()
feature_list = ['RMS', 'MAV']

In [50]:
# Crear un diccionario para almacenar las características extraídas
features_data = {}

# Iterar sobre cada sujeto en el diccionario emg_data
for key, emg in emg_data.items():
    features_data[key] = fe.extract_features(feature_list, emg)

# Mostrar las claves del diccionario de características extraídas
features_data.keys()

dict_keys(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '1', '20', '21', '22', '23', '24', '25', '26', '27', '2', '3', '4', '5', '6', '7', '8', '9'])

Verifiquemos esto para el usuario `1`

In [51]:
features_data['1'].keys()  # Mostrar las claves de las características extraídas para el sujeto 1

dict_keys(['RMS', 'MAV'])

Por ahora solo se va a realizar el analisis para el RMS:

In [52]:
features_data['1']['RMS'].shape

(801, 10)

4. Obtener los restimulos asociados a cada segmento teniendo en cuenta lo siguiente:
   * Si todos los valores de las muestras tienen el mismo valor de `restimulus`, se coloca para ese segmento dicho valor.
   * Si el segmento tiene diferentes valores de `restimulus`, es por que hay un cambio de nivel en el segmento, por lo tanto, este se marca con -1.

In [53]:
# Crear un nuevo diccionario para almacenar los resultados procesados
processed_restimulus_data = {}

# Iterar sobre cada clave y array en restimulus_data
for key, data in restimulus_data.items():
    # Crear un array donde cada fila tiene el valor repetido si todos los valores son iguales, o -1 si no lo son
    processed_restimulus_data[key] = np.where(np.all(data == data[:, [0]], axis=1), data[:, 0], -1)

# Mostrar un ejemplo del resultado procesado para una clave
print(processed_restimulus_data['1'])

[ 0.  0. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1. -1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [54]:
processed_restimulus_data['1'].shape

(801,)

5. Concatenar los valore de las caracteristicas con los `restimulos` asociados a cada segmento:

In [55]:
# Crear un diccionario para almacenar los datos concatenados
concatenated_data = {}

# Iterar sobre cada sujeto
for key in features_data.keys():
    # Concatenar las características RMS con los datos procesados de restimulus
    concatenated_data[key] = np.hstack((features_data[key]['RMS'], processed_restimulus_data[key].reshape(-1, 1)))

# Mostrar la forma de los datos concatenados para cada sujeto
for key, data in concatenated_data.items():
    print(f"Sujeto {key}: {data.shape}")

Sujeto 10: (798, 11)
Sujeto 11: (804, 11)
Sujeto 12: (796, 11)
Sujeto 13: (799, 11)
Sujeto 14: (792, 11)
Sujeto 15: (796, 11)
Sujeto 16: (799, 11)
Sujeto 17: (801, 11)
Sujeto 18: (798, 11)
Sujeto 19: (805, 11)
Sujeto 1: (801, 11)
Sujeto 20: (798, 11)
Sujeto 21: (804, 11)
Sujeto 22: (799, 11)
Sujeto 23: (794, 11)
Sujeto 24: (802, 11)
Sujeto 25: (800, 11)
Sujeto 26: (807, 11)
Sujeto 27: (794, 11)
Sujeto 2: (798, 11)
Sujeto 3: (797, 11)
Sujeto 4: (799, 11)
Sujeto 5: (807, 11)
Sujeto 6: (802, 11)
Sujeto 7: (797, 11)
Sujeto 8: (802, 11)
Sujeto 9: (805, 11)


Finalmente, vamos a verificar que los resultados tengan sentido.

In [56]:
# Obtener los datos concatenados del sujeto 1
data_subject_1 = concatenated_data['1']

# Crear un DataFrame a partir de los datos concatenados
columns = [f'RMS_{i+1}' for i in range(data_subject_1.shape[1] - 1)] + ['restimulus']
df_subject_1 = pd.DataFrame(data_subject_1, columns=columns)

# Mostrar las primeras filas del DataFrame
df_subject_1.head()

,RMS_1,RMS_2,RMS_3,RMS_4,RMS_5,RMS_6,RMS_7,RMS_8,RMS_9,RMS_10,restimulus
0,0.004408,0.000423,0.001968,0.002506,0.003413,0.023939,0.206102,0.018440,0.000678,0.074678,0.0
1,0.023491,0.005878,0.001462,0.002629,0.005496,0.037188,0.267408,0.043440,0.001412,0.142138,0.0
2,0.061386,0.026114,0.002602,0.002674,0.007024,0.044328,0.322454,0.122883,0.011375,0.215927,-1.0
3,0.084903,0.062461,0.009608,0.002956,0.006360,0.053774,0.387859,0.289713,0.037026,0.282858,1.0
4,0.061730,0.104530,0.028248,0.005728,0.007340,0.071548,0.467802,0.534735,0.086235,0.336115,1.0


Con esto se ve que el resultado tiene sentido.

##### 5.4.2. Generalización del procedimiendo de extracción de caracteristicas

Antes de empezar retomemos los datos que se tienen como punto de partida:

In [58]:
# Listar y cargar los archivos .npy del directorio PROCESSED_DATA_DIR
npy_files = [f for f in os.listdir(FILTERED_SUBSET_DATA_DIR) if f.endswith('.npy')]

data = {}
for file in npy_files:
    file_path = os.path.join(FILTERED_SUBSET_DATA_DIR, file)
    sub_i = file.split('_s')[-1].split('.')[0]
    sub_i = sub_i.split('_')[0]
    data[sub_i] = np.load(file_path)
    print(f"Archivo cargado: {file}")

Archivo cargado: filtered_subset_s10_p1_23_E3.npy
Archivo cargado: filtered_subset_s11_p1_23_E3.npy
Archivo cargado: filtered_subset_s12_p1_23_E3.npy
Archivo cargado: filtered_subset_s13_p1_23_E3.npy
Archivo cargado: filtered_subset_s14_p1_23_E3.npy
Archivo cargado: filtered_subset_s15_p1_23_E3.npy
Archivo cargado: filtered_subset_s16_p1_23_E3.npy
Archivo cargado: filtered_subset_s17_p1_23_E3.npy
Archivo cargado: filtered_subset_s18_p1_23_E3.npy
Archivo cargado: filtered_subset_s19_p1_23_E3.npy
Archivo cargado: filtered_subset_s1_p1_23_E3.npy
Archivo cargado: filtered_subset_s20_p1_23_E3.npy
Archivo cargado: filtered_subset_s21_p1_23_E3.npy
Archivo cargado: filtered_subset_s22_p1_23_E3.npy
Archivo cargado: filtered_subset_s23_p1_23_E3.npy
Archivo cargado: filtered_subset_s24_p1_23_E3.npy
Archivo cargado: filtered_subset_s25_p1_23_E3.npy
Archivo cargado: filtered_subset_s26_p1_23_E3.npy
Archivo cargado: filtered_subset_s27_p1_23_E3.npy
Archivo cargado: filtered_subset_s2_p1_23_E3.npy
Ar

**Diccionario con los datos**

In [59]:
print(type(data))
print(len(data))
print(data.keys())

<class 'dict'>
27
dict_keys(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '1', '20', '21', '22', '23', '24', '25', '26', '27', '2', '3', '4', '5', '6', '7', '8', '9'])


**Dataframe donde se concatenan todos los datos de **`data`**

In [60]:
print(type(all_users_data))
print(all_users_data.shape)


<class 'numpy.ndarray'>
(432383, 13)


In [61]:
df_all_users.sample(9)  # Mostrar una muestra aleatoria de 5 filas del DataFrame

,subject,emg_1,emg_2,emg_3,emg_4,emg_5,emg_6,emg_7,emg_8,emg_9,emg_10,restimulus,rerepetition
168972,1.0,0.003127,0.349143,0.133193,0.084101,0.126605,0.412473,1.017275,2.122241,0.554147,0.520487,23.0,2.0
226761,23.0,0.003017,1.245100,0.508007,1.017056,0.309503,0.353393,1.720029,0.469929,0.423712,0.150973,1.0,4.0
366550,5.0,0.092691,0.002371,0.002638,0.002550,0.002494,0.002471,0.002118,0.051506,0.002390,0.002469,0.0,0.0
314893,2.0,0.002953,0.755097,0.300376,0.008140,0.002147,0.001805,0.716398,1.932608,0.220691,0.279965,23.0,4.0
103403,16.0,-0.014843,-0.003239,-0.001289,0.003382,0.003814,0.003955,-0.025574,-0.002685,-0.011274,-0.031537,0.0,0.0
350898,4.0,0.280149,0.184471,0.159485,0.002813,0.028179,0.087247,0.244545,1.042085,0.188757,0.153328,23.0,9.0
207808,21.0,1.302786,0.339653,0.133918,0.111109,0.050727,0.100621,0.771256,1.286402,1.780144,0.234107,23.0,10.0
408772,8.0,0.024356,0.002394,0.002441,0.002398,0.002420,0.002462,0.002661,0.030224,0.002388,0.002453,0.0,0.0
379377,6.0,0.697504,0.032310,0.018529,0.002477,0.002600,0.016698,0.125896,0.569860,0.002392,0.025430,0.0,0.0


`df_all_users` es donde se tiene toda la información de las señales unificadas para todos los usuarios unificada y `data` es el diccionario que almacena los datos de todos los usuarios.

In [64]:
windows_data = segmentate_data_base(data, 30, 20)

In [65]:
# Mostrar las formas de las ventanas generadas para cada sujeto
for key, windows in windows_data.items():
    print(f"Sujeto {key}: {windows.shape}")

Sujeto 10: (798, 12, 30)
Sujeto 11: (804, 12, 30)
Sujeto 12: (796, 12, 30)
Sujeto 13: (799, 12, 30)
Sujeto 14: (792, 12, 30)
Sujeto 15: (796, 12, 30)
Sujeto 16: (799, 12, 30)
Sujeto 17: (801, 12, 30)
Sujeto 18: (798, 12, 30)
Sujeto 19: (805, 12, 30)
Sujeto 1: (801, 12, 30)
Sujeto 20: (798, 12, 30)
Sujeto 21: (804, 12, 30)
Sujeto 22: (799, 12, 30)
Sujeto 23: (794, 12, 30)
Sujeto 24: (802, 12, 30)
Sujeto 25: (800, 12, 30)
Sujeto 26: (807, 12, 30)
Sujeto 27: (794, 12, 30)
Sujeto 2: (798, 12, 30)
Sujeto 3: (797, 12, 30)
Sujeto 4: (799, 12, 30)
Sujeto 5: (807, 12, 30)
Sujeto 6: (802, 12, 30)
Sujeto 7: (797, 12, 30)
Sujeto 8: (802, 12, 30)
Sujeto 9: (805, 12, 30)


In [78]:
features_dict = get_features(data, ['RMS','WL','IAV'], window_size=30, window_increment=20)
features_dict.keys()  # Mostrar las claves de las características extraídas para el sujeto 1

dict_keys(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '1', '20', '21', '22', '23', '24', '25', '26', '27', '2', '3', '4', '5', '6', '7', '8', '9'])

In [79]:
for key in features_dict:
    print(f"Sujeto {key}: {features_dict[key].shape}")


Sujeto 10: (798, 31)
Sujeto 11: (804, 31)
Sujeto 12: (796, 31)
Sujeto 13: (799, 31)
Sujeto 14: (792, 31)
Sujeto 15: (796, 31)
Sujeto 16: (799, 31)
Sujeto 17: (801, 31)
Sujeto 18: (798, 31)
Sujeto 19: (805, 31)
Sujeto 1: (801, 31)
Sujeto 20: (798, 31)
Sujeto 21: (804, 31)
Sujeto 22: (799, 31)
Sujeto 23: (794, 31)
Sujeto 24: (802, 31)
Sujeto 25: (800, 31)
Sujeto 26: (807, 31)
Sujeto 27: (794, 31)
Sujeto 2: (798, 31)
Sujeto 3: (797, 31)
Sujeto 4: (799, 31)
Sujeto 5: (807, 31)
Sujeto 6: (802, 31)
Sujeto 7: (797, 31)
Sujeto 8: (802, 31)
Sujeto 9: (805, 31)


In [80]:
features_dict['10'][:1,:]

array([[1.86650220e-01, 6.49542247e-02, 7.95561316e-03, 2.36957462e-03,
        2.31630801e-03, 2.33972524e-03, 1.18192854e-02, 2.53206954e-02,
        2.16040939e-03, 2.18070240e-02, 1.06640524e-01, 7.62132102e-02,
        7.21450165e-03, 6.80126042e-05, 9.43460987e-05, 5.81239942e-04,
        3.46431812e-02, 7.96128577e-02, 3.66121578e-04, 3.52080585e-02,
        5.51080419e+00, 1.81859813e+00, 2.32948032e-01, 7.10845451e-02,
        6.94834564e-02, 6.99990608e-02, 2.58276771e-01, 6.94282062e-01,
        6.47236241e-02, 6.12146668e-01, 0.00000000e+00]])

In [87]:
features_dict.keys()
features_dict['1'].shape  # Mostrar la forma de las características extraídas para el sujeto 1

(801, 31)

### Almacenamiendo de las features

Las features se almacenan en un archivo csv:

Vemos que el resultado de lo anterior es in diccionario con las caracteristicas por usuario

In [131]:
FEATURES_SUBSET_DATA_DIR

'./local/data/features_subset'

In [132]:
save_features_to_csv(
    features_dict = features_dict,
    feature_list= ['RMS','WL','IAV'],
    output_dir = FEATURES_SUBSET_DATA_DIR,
    label_column_name='restimulus'
)

Guardado: ./local/data/features_subset\sujeto_10_features.csv
Guardado: ./local/data/features_subset\sujeto_11_features.csv
Guardado: ./local/data/features_subset\sujeto_12_features.csv
Guardado: ./local/data/features_subset\sujeto_13_features.csv
Guardado: ./local/data/features_subset\sujeto_14_features.csv
Guardado: ./local/data/features_subset\sujeto_15_features.csv
Guardado: ./local/data/features_subset\sujeto_16_features.csv
Guardado: ./local/data/features_subset\sujeto_17_features.csv
Guardado: ./local/data/features_subset\sujeto_18_features.csv
Guardado: ./local/data/features_subset\sujeto_19_features.csv
Guardado: ./local/data/features_subset\sujeto_1_features.csv
Guardado: ./local/data/features_subset\sujeto_20_features.csv
Guardado: ./local/data/features_subset\sujeto_21_features.csv
Guardado: ./local/data/features_subset\sujeto_22_features.csv
Guardado: ./local/data/features_subset\sujeto_23_features.csv
Guardado: ./local/data/features_subset\sujeto_24_features.csv
Guardado:

## Conclusiones

Al finalizar este notebook se logro.
* Se logro aplicar la segmentación y comprender el formato de lo datos arrojados en la matriz asociada a los segmentos.
* Se obtuvo el valor RMS de todo.
* Se mostro un dataframe con el valor RMS para el sujeto 1.
* En teoria se pudo guardar los datos asociados al sujeto en cuestion.